<a href="https://colab.research.google.com/github/YichengShen/cis5220-project/blob/main/cis5220_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text-to-SQL

Team: Query Marksman

## Section 0: Config

Flags for choosing which model to run

In [1]:
RUN_NON_DL_MODEL = True

Seed

In [2]:
SEED = 0

## Section 1: Setup

### Install & imports

In [3]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import shutil
import subprocess

import re
import io
import json
import numpy as np
import os
import nltk
from typing import List, Dict, Tuple, Any, Union

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Mount Drive

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Load data into Colab notebook

Before you run the code below, make sure you download the Spider dataset from [here](https://yale-lily.github.io/spider). Then, you upload the zip file of the dataset to your Drive.

Copy data from Drive into the current runtime

In [7]:
# Create data folder if not exist
!mkdir -p data

# Change this path to where you store spider.zip in your Drive
dataset_zip_path_in_drive = "/content/drive/MyDrive/CIS5220_final_project/spider.zip"
dataset_zip_path_in_runtime = "/content/data/spider.zip"

shutil.copy(dataset_zip_path_in_drive, dataset_zip_path_in_runtime)

'/content/data/spider.zip'

Unzip

In [8]:
!unzip -q -o /content/data/spider.zip -d /content/data/

### Load Python scripts

In [9]:
scripts_path_in_drive = "/content/drive/MyDrive/CIS5220_final_project/scripts"
scripts_path_in_runtime = "/content/scripts"

# Overrides previous scripts folder
if os.path.exists(scripts_path_in_runtime):
    shutil.rmtree(scripts_path_in_runtime)
shutil.copytree(scripts_path_in_drive, scripts_path_in_runtime)

'/content/scripts'

## Section 2: Data Preparation & Cleaning (Milestone 1)

### Helper Functions

In [10]:
def process(sql_data: List[Dict], 
            table_data: List[Dict]) -> Tuple[List[Dict], Dict[str, Dict]]:
    output_tab = {}
    for i in range(len(table_data)):
      table = table_data[i]
      temp = {}
      temp['col_map'] = table['column_names']

      db_name = table['db_id']
      output_tab[db_name] = temp


    output_sql = []
    for i in range(len(sql_data)):
      sql = sql_data[i]
      temp = {}

      # add query metadata
      temp['question'] = sql['question']
      temp['question_tok'] = sql['question_toks']
      temp['query'] = sql['query'].replace('\t', '') # Remove \t, this affects generating the label file
      temp['query_tok'] = sql['query_toks']
      temp['table_id'] = sql['db_id']
      sql_temp = {}

      # process agg/sel
      sql_temp['agg'] = []
      sql_temp['sel'] = []
      gt_sel = sql['sql']['select'][1]
      for tup in gt_sel:
        sql_temp['agg'].append(tup[0])
        sql_temp['sel'].append(tup[1][1][1])
      
      # process where conditions and conjuctions
      sql_temp['cond'] = []
      gt_cond = sql['sql']['where']
      if len(gt_cond) > 0:
        conds = [gt_cond[x] for x in range(len(gt_cond)) if x % 2 == 0]
        for cond in conds:
          curr_cond = []
          curr_cond.append(cond[2][1][1])
          curr_cond.append(cond[1])
          if cond[4] is not None:
            curr_cond.append([cond[3], cond[4]])
          else:
            curr_cond.append(cond[3])
          sql_temp['cond'].append(curr_cond)

      sql_temp['conj'] = [gt_cond[x] for x in range(len(gt_cond)) if x % 2 == 1]

      # process group by / having
      sql_temp['group'] = [x[1] for x in sql['sql']['groupBy']]
      having_cond = []
      if len(sql['sql']['having']) > 0:
        gt_having = sql['sql']['having'][0] # currently only do first having condition
        having_cond.append(gt_having[2][1][0]) # aggregator
        having_cond.append(gt_having[2][1][1]) # column
        having_cond.append(gt_having[1]) # operator
        if gt_having[4] is not None:
          having_cond.append([gt_having[3], gt_having[4]])
        else:
          having_cond.append(gt_having[3])
      sql_temp['group'].append(having_cond)

      # process order by / limit
      order_aggs = []
      order_cols = []
      order_par = -1
      gt_order = sql['sql']['orderBy']
      if len(gt_order) > 0:
        order_aggs = [x[1][0] for x in gt_order[1]]
        order_cols = [x[1][1] for x in gt_order[1]]
        order_par = 1 if gt_order[0] == 'asc' else 0
      sql_temp['order'] = [order_aggs, order_cols, order_par]

      # process intersect/except/union
      sql_temp['special'] = 0
      if sql['sql']['intersect'] is not None:
        sql_temp['special'] = 1
      elif sql['sql']['except'] is not None:
        sql_temp['special'] = 2
      elif sql['sql']['union'] is not None:
        sql_temp['special'] = 3

      temp['sql'] = sql_temp
      output_sql.append(temp)
    return output_sql, output_tab

In [11]:
def load_data_new(sql_paths: Union[str, List[str]], 
                  table_paths: Union[str, List[str]], 
                  use_small: bool = False) -> Tuple[List[Dict], Dict[str, Dict]]:
    if not isinstance(sql_paths, list):
        sql_paths = (sql_paths, )
    if not isinstance(table_paths, list):
        table_paths = (table_paths, )
    sql_data = []
    table_data = {}
    for i, SQL_PATH in enumerate(sql_paths):
        if use_small and i >= 2:
            break
        print(f"Loading data from {SQL_PATH}")
        with open(SQL_PATH) as inf:
            data = json.load(inf)
            sql_data += data
                
    for i, TABLE_PATH in enumerate(table_paths):
        if use_small and i >= 2:
            break
        print(f"Loading data from {TABLE_PATH}")
        with open(TABLE_PATH) as inf:
            table_data= json.load(inf)
    # print sql_data[0]
    sql_data, table_data = process(sql_data, table_data)
    return sql_data, table_data

### Load Clean Data

In [12]:
sql_data_train, table_data = load_data_new(["/content/data/spider/train_spider.json"], ["/content/data/spider/tables.json"], use_small=False)

Loading data from /content/data/spider/train_spider.json
Loading data from /content/data/spider/tables.json


In [13]:
sql_data_dev, table_data = load_data_new(["/content/data/spider/dev.json"], ["/content/data/spider/tables.json"], use_small=False)

Loading data from /content/data/spider/dev.json
Loading data from /content/data/spider/tables.json


## Section 3: EDA

### SQL Data

In [14]:
print(f"Number of training data: {len(sql_data_train)}")
print(f"Number of eval data: {len(sql_data_dev)}")

Number of training data: 7000
Number of eval data: 1034


One example looks like:

In [15]:
sql_data_train[0]

{'question': 'How many heads of the departments are older than 56 ?',
 'question_tok': ['How',
  'many',
  'heads',
  'of',
  'the',
  'departments',
  'are',
  'older',
  'than',
  '56',
  '?'],
 'query': 'SELECT count(*) FROM head WHERE age  >  56',
 'query_tok': ['SELECT',
  'count',
  '(',
  '*',
  ')',
  'FROM',
  'head',
  'WHERE',
  'age',
  '>',
  '56'],
 'table_id': 'department_management',
 'sql': {'agg': [3],
  'sel': [0],
  'cond': [[10, 3, 56.0]],
  'conj': [],
  'group': [[]],
  'order': [[], [], -1],
  'special': 0}}

### Database Schema Data

In [16]:
len(table_data)

166

In [17]:
table_data['yelp']

{'col_map': [[-1, '*'],
  [0, 'bid'],
  [0, 'business id'],
  [0, 'name'],
  [0, 'full address'],
  [0, 'city'],
  [0, 'latitude'],
  [0, 'longitude'],
  [0, 'review count'],
  [0, 'is open'],
  [0, 'rating'],
  [0, 'state'],
  [1, 'id'],
  [1, 'business id'],
  [1, 'category name'],
  [2, 'uid'],
  [2, 'user id'],
  [2, 'name'],
  [3, 'cid'],
  [3, 'business id'],
  [3, 'count'],
  [3, 'day'],
  [4, 'id'],
  [4, 'business id'],
  [4, 'neighbourhood name'],
  [5, 'rid'],
  [5, 'business id'],
  [5, 'user id'],
  [5, 'rating'],
  [5, 'text'],
  [5, 'year'],
  [5, 'month'],
  [6, 'tip id'],
  [6, 'business id'],
  [6, 'text'],
  [6, 'user id'],
  [6, 'likes'],
  [6, 'year'],
  [6, 'month']]}

## Section 4: Code for Evaluation

In [18]:
def evaluate(preds_file, labels_file, evaluation_type="all", 
             database_dir="./data/spider/database", 
             table_file="./data/spider/tables.json",
             verbose="False"):
    """
    Runs the evaluation script for the Spider dataset using the provided labels and predictions files.
    It prints the evaluation results to the console and returns the subprocess result object.

    Args:
        preds_file (str): Path to the predictions file. In this file, each line is `a ground-truth SQL \t db_id`.
        labels_file (str): Path to the labels (gold) file. In this file, each line is a predicted SQL.
        evaluation_type (str): Evaluation type, can be 'all', 'exec', or 'match'.
        database_dir (str): Path to the directory containing the Spider dataset's database files.
        table_file (str): Path to the tables.json file from the Spider dataset.
        verbose (str): Flag to trun on or off printing details.

    Returns:
        result (subprocess.CompletedProcess): A CompletedProcess instance representing the evaluation subprocess.
                                              It contains attributes like 'stdout' and 'stderr' to access the output
                                              and error messages respectively.
    """

    cmd = [
        "python3", "scripts/evaluation.py",
        "--gold", labels_file,
        "--pred", preds_file,
        "--etype", evaluation_type,
        "--db", database_dir,
        "--table", table_file,
        "--verbose", verbose
    ]

    result = subprocess.run(cmd, capture_output=True, text=True)

    print(result.stdout)

    return result

## Section 5: Non-DL Model (Milestone 2)

In [19]:
import json
import numpy as np
import torch
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

### Random Forest Model

In [20]:
class RF_TextToSQL(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.vectorizer = HashingVectorizer(n_features=2**16)
        self.model = RandomForestClassifier(n_estimators=60)
        self.label_encoder = LabelEncoder()

    def fit(self, X, y):
        y_encoded = self.label_encoder.fit_transform(y)
        X_vec = self.vectorizer.transform(X)
        self.model.fit(X_vec, y_encoded)

    def predict(self, X):
        X_vec = self.vectorizer.transform(X)
        y_pred_encoded = self.model.predict(X_vec)
        return self.label_encoder.inverse_transform(y_pred_encoded)

    def write_predictions_to_file(self, text_questions, labels, preds_filename, labels_filename):
        predicted_sql_queries = self.predict(text_questions)

        with open(preds_filename, 'w') as output_file:
            for pred in predicted_sql_queries:
                output_file.write(pred + '\n')

        with open(labels_filename, 'w') as output_file:
            for label in labels:
                output_file.write(label + '\n')
                

def sql_dict_to_string(sql_dict):
    sql_string = json.dumps(sql_dict, separators=(',', ':'))
    return sql_string

def get_text_and_sql(data):
    text = []
    sql = []
    sql_with_table_id = []

    for item in data:
        text.append(item['question'])
        sql.append(item['query'])
        sql_with_table_id.append(f"{item['query']}\t{item['table_id']}")
        # sql.append(sql_dict_to_string(item['sql']))

    return text, sql, sql_with_table_id

In [21]:
if RUN_NON_DL_MODEL:
    # Preprocess data
    train_text, train_sql, train_sql_with_table_id = get_text_and_sql(sql_data_train)
    dev_text, dev_sql, dev_sql_with_table_id = get_text_and_sql(sql_data_dev)

    # Use a smaller subset of the dataset
    SUBSET_SIZE = 0.99
    _, _, train_sql_with_table_id, _ = train_test_split(train_text, train_sql_with_table_id, train_size=SUBSET_SIZE, random_state=SEED)
    train_text, _, train_sql, _ = train_test_split(train_text, train_sql, train_size=SUBSET_SIZE, random_state=SEED)

    # Train-test split
    TEST_SIZE = 0.3
    _, _, y_train_with_table_id, y_val_with_table_id = train_test_split(train_text, train_sql_with_table_id, test_size=TEST_SIZE, random_state=SEED)
    X_train, X_val, y_train, y_val = train_test_split(train_text, train_sql, test_size=TEST_SIZE, random_state=SEED)

    # Initialize model
    model = RF_TextToSQL()

    # Train model
    model.fit(X_train, y_train)

### Evaluation

Evaluation on the training set

In [22]:
# Write predictions of training set to a file
model.write_predictions_to_file(X_train, y_train_with_table_id, 'preds.txt', 'labels.txt')

evaluation_train = evaluate(preds_file="preds.txt", 
                            labels_file="labels.txt", 
                            evaluation_type="all", 
                            database_dir="./data/spider/database", 
                            table_file="./data/spider/tables.json",
                            verbose="False")

                     easy                 medium               hard                 extra                all                 
count                1159                 1943                 1001                 748                  4851                
=====================   EXECUTION ACCURACY     =====================
execution            0.997                0.999                1.000                0.999                0.999               

====================== EXACT MATCHING ACCURACY =====================
exact match          0.997                0.999                1.000                1.000                0.999               

---------------------PARTIAL MATCHING ACCURACY----------------------
select               1.000                1.000                1.000                1.000                1.000               
select(no AGG)       1.000                1.000                1.000                1.000                1.000               
where                1.000         

Evaluation on the test set

In [23]:
model.write_predictions_to_file(dev_text, dev_sql_with_table_id, 'preds.txt', 'labels.txt')

evaluation_test = evaluate(preds_file="preds.txt", 
                          labels_file="labels.txt", 
                          evaluation_type="all", 
                          database_dir="./data/spider/database", 
                          table_file="./data/spider/tables.json",
                          verbose="False")

                     easy                 medium               hard                 extra                all                 
count                248                  446                  174                  166                  1034                
=====================   EXECUTION ACCURACY     =====================
execution            0.012                0.000                0.000                0.000                0.003               

====================== EXACT MATCHING ACCURACY =====================
exact match          0.012                0.000                0.000                0.000                0.003               

---------------------PARTIAL MATCHING ACCURACY----------------------
select               0.556                0.091                0.000                0.333                0.269               
select(no AGG)       0.556                0.091                0.000                0.333                0.269               
where                0.000         